In [1]:
import re
import os
import math
import torch
import gpytorch
import itertools
import numpy as np
import pandas as pd
from torch import nn
import os.path as op
from tqdm import tqdm
from time import time
from copy import deepcopy
from torch.optim import Adam
from collections import Counter
import torch.nn.functional as F
import matplotlib.pyplot as plt
from capsules.capsules import *
from CoordConv.coordconv import *
from IPython.display import Image
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score
from torchvision import models, transforms
from scipy.stats import pearsonr, spearmanr
from gpytorch.priors import SmoothedBoxPrior
from torch.utils.data import DataLoader, Dataset
from gpytorch.models import ApproximateGP, ExactGP
from gpytorch.means import ConstantMean, LinearMean
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.variational import VariationalStrategy
from gpytorch.distributions import MultivariateNormal
from Bio.SeqFeature import SeqFeature, FeatureLocation
from gpytorch.models.deep_gps import DeepGPLayer, DeepGP
from gpytorch.variational import CholeskyVariationalDistribution
from sklearn.model_selection import train_test_split, StratifiedKFold
from gpytorch.kernels import ScaleKernel, RBFKernel, GridInterpolationKernel
from gpytorch.mlls import VariationalELBO, VariationalELBOEmpirical
from gpytorch.mlls import DeepApproximateMLL
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import median_absolute_error


/home/bakirillov/anaconda3/envs/lapki/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [22]:
from core import ImperfectMatchTransform

In [17]:
class FlashFrySet(Dataset):
    
    def __init__(self, dataframe, transform=None):
        self.S = pd.read_csv(dataframe, sep="\t")
        self.T = transform
        if transform:
            self.S = FlashFrySet.get_pairs(self.S)
        
    @staticmethod
    def get_pairs(d):
        pairs = []
        for a in d.index:
            target = d.loc[a]["target"]
            ots = d.loc[a]["offTargets"].split(",")
            for ot in ots:
                pairs.append(
                    target+","+ot.split("_")[0]
                )
        return(pairs)
        
        
    def __len__(self):
        if self.T:
            return(len(S))
        else:
            return(self.S.shape[0]-1)
    
    def __getitem__(self, ind):
        if not self.T:
            guide = self.S["target"].values[ind]
            otCount = self.S["otCount"].values[ind]
            return(guide, otCount)
        else:
            pair = self.T(self.S[ind])
            return(pair)

In [23]:
T = ImperfectMatchTransform("NGG", False, False, cut_at_start=0, cut_at_end=0, fold=False)

In [24]:
ff = FlashFrySet("../../webapp/crispr_cad/backend/tmp/49345.tsv", transform=T)

In [27]:
for a in ff:
    print(a.shape)

(2, 4, 23)
(2, 4, 23)
(2, 4, 23)
(2, 4, 23)
(2, 4, 23)
(2, 4, 23)
(2, 4, 23)
(2, 4, 23)
(2, 4, 23)
(2, 4, 23)
(2, 4, 23)


In [7]:
test

,contig,start,stop,target,context,overflow,orientation,otCount,offTargets
0,87,0,23,TTTTCGAGGACTGAGAGGGAGGG,NONE,OK,FWD,1,TTTTCTAGGAATGAAAAGGAAGG_1_4
1,211,0,23,CTTGTGACACGGACTCAAGTGGG,NONE,OK,FWD,1,TTTGTGACAAGGACTCTCGTGGG_1_4
2,78,0,23,TGTCATCTATGACCTTCCCTGGG,NONE,OK,FWD,6,"AGTCATATATCACCTTCTCTTGG_1_4,AGTCATCCATGTTCTTCC..."
3,186,0,23,GGTGTCGAAATGAGAAGAAGAGG,NONE,OK,FWD,3,"CATGTTGAAATGAGAAGAATTGG_1_4,GGTGGCAACATGAGAAGA..."


In [14]:
test.loc[2]["offTargets"].split(",")

['AGTCATATATCACCTTCTCTTGG_1_4',
 'AGTCATCCATGTTCTTCCCTAGG_1_4',
 'TCTCAGCCATGGCCTTCCCTGGG_1_4',
 'TGCCATCTATCGCATTCCCTGGG_1_4',
 'TGTCATCTGTGACATTACCCAGG_1_4',
 'TGTCTTTTATGACCTTGCCTTGG_1_3']

In [15]:
def get_pairs(d):
    pairs = []
    for a in d.index:
        target = d.loc[a]["target"]
        ots = d.loc[a]["offTargets"].split(",")
        for ot in ots:
            pairs.append(
                target+","+ot.split("_")[0]
            )
    return(pairs)

In [16]:
get_pairs(test)

['TTTTCGAGGACTGAGAGGGAGGG,TTTTCTAGGAATGAAAAGGAAGG',
 'CTTGTGACACGGACTCAAGTGGG,TTTGTGACAAGGACTCTCGTGGG',
 'TGTCATCTATGACCTTCCCTGGG,AGTCATATATCACCTTCTCTTGG',
 'TGTCATCTATGACCTTCCCTGGG,AGTCATCCATGTTCTTCCCTAGG',
 'TGTCATCTATGACCTTCCCTGGG,TCTCAGCCATGGCCTTCCCTGGG',
 'TGTCATCTATGACCTTCCCTGGG,TGCCATCTATCGCATTCCCTGGG',
 'TGTCATCTATGACCTTCCCTGGG,TGTCATCTGTGACATTACCCAGG',
 'TGTCATCTATGACCTTCCCTGGG,TGTCTTTTATGACCTTGCCTTGG',
 'GGTGTCGAAATGAGAAGAAGAGG,CATGTTGAAATGAGAAGAATTGG',
 'GGTGTCGAAATGAGAAGAAGAGG,GGTGGCAACATGAGAAGAAGTGG',
 'GGTGTCGAAATGAGAAGAAGAGG,GGTGTTGCAGTCAGAAGAAGGGG']